In [78]:
import importlib
import SpotifyAPIClient
import pprint
import csv
import os
import random
importlib.reload(SpotifyAPIClient)
from SpotifyAPIClient import SpotifyAPI
import pandas as pd

import mysklearn.myutils
importlib.reload(mysklearn.myutils)
import mysklearn.myutils as myutils

import mysklearn.mypytable
importlib.reload(mysklearn.mypytable)
from mysklearn.mypytable import MyPyTable 

import mysklearn.myevaluation
importlib.reload(mysklearn.myevaluation)
import mysklearn.myevaluation as myevaluation

import mysklearn.myclassifiers
importlib.reload(mysklearn.myclassifiers)
from mysklearn.myclassifiers import MyKNeighborsClassifier, MyNaiveBayesClassifier, MyDecisionTreeClassifier

fname = os.path.join("data", "key-track-audio-features.txt")
track_data = MyPyTable().load_from_file(fname)


client_id = 'd7a2e6f4a8434550baa5eda073f0a6a3'
client_secret = 'def46d47ba584e378b7667645666a468'
playlist_id = '7L736vCRhBe5EapwwkutUl'

In [79]:
spotify = SpotifyAPI(client_id, client_secret)
response = spotify.get_playlist(playlist_id)


In [80]:
track_ids, track_popularity = spotify.parse_for_track_id(response)
#print("track_ids: ", track_ids)
print()
print()
header = ["Track Name", "Track ID", "Danceability", "Energy", "Valence", "Popularity"] 
tracks_feature_list = []

did_once = False
count = 0
features_header = []
features_header.append("song_id")
features_header.append("song_name")
songs_vals_list = [] #[song_id, song_name, danceability, energy, key, loudness, mode, speechiness, acousticness, instrumentalness, liveness, valence, tempo, type, id, uri, track_href, analysis_url, duration_ms, time_signature ]

for name, id in track_ids.items():
    #print("name: ", name, " ----- id: ", id)
    song = []
    song.append(id)
    song.append(name)

    track_feat_list = []
    response = spotify.get_track_audio_features(id)
    #print("response: ", response)
    do_not_add = ["type","id","uri","track_href","analysis_url"]

    for k, v in response.items():
        #print("     k: ", k, "      v: ", v)
        #type,id,uri,track_href,analysis_url
        if(k not in do_not_add):
            if not did_once:
                features_header.append(k)
                #print("v type: ", type(v))
            song.append(v)
    did_once = True
    song.append(track_popularity[count])
    songs_vals_list.append(song)

    track_feat_list.append(name)
    track_feat_list.append(id)
    track_feat_list.append(round(response['danceability'] * 100, 2))
    track_feat_list.append(round(response['energy'] * 100, 2))
    # track_feat_list.append(response['tempo'])
    track_feat_list.append(round(response['valence'] * 100, 2))
    #print("track_popularity[count]: ", track_popularity[count])
    track_feat_list.append(track_popularity[count])
    count += 1
    tracks_feature_list.append(track_feat_list)
features_header.append("song_popularity")

#print("features_header: ", features_header)
#print("songs_vals_list: ", songs_vals_list)
#myutils.table_val_print(features_header, songs_vals_list)
#myutils.print_tabulate(songs_vals_list, features_header)
myutils.print_tabulate(tracks_feature_list, header)


    



====================================================  ======================  ==============  ========  =========  ============
Track Name                                            Track ID                  Danceability    Energy    Valence    Popularity
====================================================  ======================  ==============  ========  =========  ============
Are We There Yet                                      39LRXm8xZ3DP2W37sl1NDj            71.1      58.1       62.1            43
Sweet Nothing (feat. Florence Welch)                  24LS4lQShWyixJ0ZrJXfJ5            57.3      92.9       58.2            73
Wish You Were Mine - Radio Edit                       412luShbmlgqqgYFStIB1s            65.4      83.2       40.7            66
All My Love (feat. Conor Maynard)                     1839RWzPRZCVgvamu4sHSN            59        87.1       30.4            60
Closer                                                7BKLCZ1jbUBVqRi2FVlTVw            74.8      52.4

In [81]:
file_name = os.path.join('data', 'key-track-audio-features.txt')
with open(file_name, 'w') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(header)
    csv_writer.writerows(tracks_feature_list)

file_name2 = os.path.join('data', 'track-audio-features-all.txt')
with open(file_name2, 'w') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(features_header)
    csv_writer.writerows(songs_vals_list)

In [82]:
# Get the file data

print("===========================================")
print("Nearest Neighbor 'Popularity' Classifier (k=5)")
print("===========================================")

mknc = MyKNeighborsClassifier(5) # set the amount of neighbors to 5

# Get the 3 column data sets
danceability = track_data.get_column('Danceability') 
energy = track_data.get_column('Energy')
valence = track_data.get_column('Valence')

# store each columns data in x_vals and scale the data
x_vals = [[danceability[i], energy[i],valence[i]] for i in range(len(danceability))]
print()
print("x_vals: ", x_vals)
scaled_x, blank = myutils.scale(x_vals, [[]])

# train and split the test list
x_train_split, x_test_split, y_train_split, y_test_split = myevaluation.train_test_split(scaled_x, 
track_data.get_column('Popularity'), shuffle=True)
# print()
# print()
# print("x_train_split: ",  x_train_split) 
# print()
# print("x_test_split: ", x_test_split) 
# print()
# print("y_train_split: ", y_train_split)
# print()
# print("y_test_split: ", y_test_split)

mknc.fit(x_train_split, y_train_split) # fit the data
# rand_rows = myutils.get_rand_rows(track_data, 5) # randomize 5 rows

# predict using the regression line
predict = mknc.predict(x_test_split)
predicted = [val for val in predict]
expected = [val for val in y_test_split]
correct = 0
for i in range(len(predicted)):
    if predicted[i] == expected[i]:
        correct += 1
linear_acc = correct / len(predicted)
print('accuracy =', linear_acc)
# predicted = [jupyter_utils.get_rating(val) for val in predicted]
# expected = [jupyter_utils.get_rating(val) for val in y_test_split]

# # for each row in the random list of 5
# for row in rand_rows:
#     print('instance:', row)
#     print("row[4]: ", row[2])
#     print("row[1]: ", row[3])
#     print("row[5]: ", row[11])
#     scaled_vals, scaled_test = myutils.scale(x_vals, [[row[2],row[3],row[11]]])
#     print("scaled_vals: ", scaled_vals)
#     print("scaled_test: ", scaled_test)
#     #print('class:', myutils.get_rating(mknc.predict(scaled_test)[0]), 'actual: ', juputils.get_rating(row[0]))

Nearest Neighbor 'Popularity' Classifier (k=5)

x_vals:  [[71.1, 58.1, 62.1], [57.3, 92.9, 58.2], [65.4, 83.2, 40.7], [59.0, 87.1, 30.4], [74.8, 52.4, 66.1], [58.4, 71.2, 28.6], [74.2, 83.8, 69.4], [78.0, 79.8, 77.1], [38.8, 75.9, 28.6], [75.7, 71.1, 64.9], [50.6, 80.5, 38.3], [32.1, 67.5, 27.4], [59.6, 85.4, 15.2], [73.8, 81.4, 48.3], [78.4, 75.7, 58.7], [61.0, 84.1, 43.1], [66.4, 70.8, 74.2], [64.6, 82.3, 41.8], [78.8, 60.4, 16.0], [75.3, 71.7, 85.9], [71.5, 58.1, 37.8], [70.1, 69.5, 62.4], [73.7, 66.7, 65.8], [73.3, 71.0, 96.5], [66.4, 70.5, 47.7], [57.8, 86.2, 54.7], [71.9, 84.0, 63.6], [81.8, 70.5, 77.2], [49.4, 84.2, 57.4], [43.0, 39.2, 12.7], [76.6, 67.4, 21.4], [80.3, 49.9, 74.9], [67.0, 85.5, 64.8], [68.7, 71.0, 88.6], [44.2, 83.0, 57.8], [67.0, 66.8, 60.8], [84.9, 48.3, 28.2], [71.5, 63.5, 39.0], [62.9, 82.8, 51.2], [67.4, 74.0, 43.1], [64.0, 64.8, 54.8], [67.4, 77.4, 33.0], [75.9, 60.5, 64.0], [62.5, 81.4, 53.5], [68.6, 71.8, 55.7], [65.6, 61.5, 36.6], [66.3, 78.3, 62.2], [6

In [83]:
danceability = track_data.get_column('Danceability') 
energy = track_data.get_column('Energy')
valence = track_data.get_column('Valence')

x_vals = [[danceability[i], energy[i], valence[i]] for i in range(len(danceability))]
y_vals = myutils.convert_to_rank(track_data.get_column("Popularity"))
print(y_vals)

print('Stratified 10-Fold Cross Validation\n')
strat_train_folds, strat_test_folds = myevaluation.stratified_kfold_cross_validation(x_vals, y_vals, 10)
strat_xtrain, strat_ytrain, strat_xtest, strat_ytest = myutils.get_from_folds(x_vals, y_vals, strat_train_folds, strat_test_folds)

myNb = MyNaiveBayesClassifier()
myNb.fit(strat_xtrain, strat_ytrain)

predicted = myNb.predict(strat_xtest)
accuracy = myutils.get_accuracy(strat_ytest, predicted)
print(strat_ytest)
print(predicted)
print(accuracy)

[5, 8, 7, 6, 9, 6, 6, 7, 5, 6, 8, 5, 8, 6, 7, 7, 8, 8, 3, 4, 4, 5, 5, 9, 8, 6, 5, 9, 8, 3, 6, 7, 8, 7, 6, 8, 5, 8, 7, 6, 9, 9, 8, 2, 8, 7, 7, 8]
Stratified 10-Fold Cross Validation

[5, 9, 8, 3, 9, 5, 6, 8, 6, 9, 5, 6, 3, 7, 8, 5, 7, 4, 8, 2, 5, 6, 4, 7, 8, 5, 8, 9, 6, 7, 5, 8, 8, 8, 7, 8, 6, 6, 8, 8, 7, 7, 9, 7, 6, 7, 8, 6]
[5, 9, 8, 3, 9, 5, 6, 8, 6, 9, 5, 6, 3, 7, 8, 5, 7, 4, 8, 2, 5, 6, 4, 7, 8, 5, 8, 9, 6, 7, 5, 8, 8, 8, 7, 8, 6, 6, 8, 8, 7, 7, 9, 7, 6, 7, 8, 6]
1.0


In [84]:
strat_train_folds2, strat_test_folds2 = myevaluation.stratified_kfold_cross_validation(x_vals, y_vals, 10)
strat_xtrain2, strat_ytrain2, strat_xtest2, strat_ytest2 = myutils.get_from_folds(x_vals, y_vals, strat_train_folds2, strat_test_folds2)

my_tree = MyDecisionTreeClassifier()
my_tree.fit(strat_xtrain2, strat_ytrain2)

predicted2 = my_tree.predict(strat_xtest2)
accuracy2 = myutils.get_accuracy(strat_ytest2, predicted2)
print('My Decision Tree: accuracy =', accuracy2, 'error rate = ', (1-accuracy2))

My Decision Tree: accuracy = 0.8958333333333334 error rate =  0.10416666666666663
